In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 10.15 GB
Used RAM: 5.05 GB
Percentage Usage Of RAM: 34.4%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 13.17 GB
Used Disk: 11.82 GB
Percentage Usage Of Disk: 47.3%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from lm_regressor import Regression

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# get the data
unemployment = pd.read_csv("unemployment.csv")
unemployment = unemployment.sort_values(by=[
    "Date", 
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
]).reset_index(drop=True)

In [5]:
# resort the data
unemployment = unemployment.sort_values(by=[
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
    "Date", 
]).reset_index(drop=True)

In [6]:
# lag the output variable
unemployment["Unemployment Rate(t-1)"] = unemployment.groupby([
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
])["Unemployment Rate "].shift(1)

unemployment["Unemployment Rate(t-2)"] = unemployment.groupby([
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
])["Unemployment Rate "].shift(2)

unemployment["Unemployment Rate(t-3)"] = unemployment.groupby([
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
])["Unemployment Rate "].shift(3)

unemployment["Unemployment Rate(t-4)"] = unemployment.groupby([
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
])["Unemployment Rate "].shift(4)

In [7]:
# remove rows with missing values
unemployment = unemployment.dropna().reset_index(drop=True)

In [8]:
# resort the data
unemployment = unemployment.sort_values(by=[
    "Date", 
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
]).reset_index(drop=True)

In [9]:
# only use 50% of the data
unemployment = unemployment.head(int(0.5 * unemployment.shape[0]))

In [10]:
# get the testing data
y = unemployment[["Unemployment Rate "]]
X = unemployment.drop(columns=["Unemployment Rate ", "Employment ", "Unemployment ", "Year ", "Month"])
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [11]:
# build the model
print("\n---- Unemployment Regression Analysis ----\n")
model = Regression(
    name="Lasso Without Feature Engineering", 
    path=None,
    rename=True, 
    time=True, 
    text=False,
    binary=True, 
    imputation=False, 
    variance=True,
    scale=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(unemployment)
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"R2: {model.r2}")
    print(f"RMSE: {model.rmse}")
    print(f"In Control: {model.in_control}")


---- Unemployment Regression Analysis ----

Visualizing The Data:
> Plotting Correlations
> Labor Force vs. Employment 
> Labor Force vs. Unemployment 
> Employment  vs. Unemployment 
> Unemployment Rate  vs. Unemployment Rate(t-1)
> Unemployment Rate(t-1) vs. Unemployment Rate(t-2)
> Unemployment Rate(t-2) vs. Unemployment Rate(t-3)
> Unemployment Rate(t-3) vs. Unemployment Rate(t-4)
> Plotting Labor Force
> Plotting Employment 
> Plotting Unemployment 
> Plotting Unemployment Rate 
> Plotting Unemployment Rate(t-1)
> Plotting Unemployment Rate(t-2)
> Plotting Unemployment Rate(t-3)
> Plotting Unemployment Rate(t-4)
> Plotting Area Type 
> Plotting Seasonally Adjusted (Y/N) 
> Area Type  vs. Month
> Area Type  vs. Seasonally Adjusted (Y/N) 
> Area Type  vs. Status (Preliminary / Final) 
> Month vs. Seasonally Adjusted (Y/N) 
> Seasonally Adjusted (Y/N)  vs. Status (Preliminary / Final) 
> Labor Force vs. Area Type 
> Labor Force vs. Area Name 
> Labor Force vs. Month
> Labor Force vs

In [12]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10].tolist()):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10].tolist()):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. Unemployment_Rate(t-1)
2. Area_Name__Prattville_CDP
3. Area_Name__Chilcoot_Vinton_CDP
4. Area_Name__Richgrove_CDP
5. Unemployment_Rate(t-4)
6. Area_Name__East_Orosi_CDP
7. Area_Name__Pajaro_CDP
8. Area_Name__Chinese_Camp_CDP
9. Area_Name__Calexico_city
10. Area_Name__Cromberg_CDP
 
Feature Drift:
1. Date_year_2019
2. Date_year_2014
3. Date_year_2016
4. Date_year_2018
5. Date_year_2023
6. Unemployment_Rate(t-1)
7. Area_Type__County
8. Date_day_of_week_Tuesday
9. Date_day_of_week_Sunday
10. Date_year_2020


In [13]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 0.039471309098275016
R2: 0.6710068952766237


In [14]:
# save the machine learning pipeline
model.dump()

In [ ]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Renaming Features
> Extracting Time Features
> Transforming Categorical Features
> Renaming Features
> Removing Constant Features
> Scaling Features
> Training Lasso
